In [1]:
import pandas as pd
import numpy as np
from data_master import DataMaster
from utils import func
from scipy.stats import norm
master = DataMaster()
from utils.func import center
from quantstats.stats import sharpe
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:

market_data = pd.read_parquet('data/US/test_mkt_data.pq')
balance_sheet = pd.read_parquet('data/US/test_balance_sheet.pq')
cash_flow = pd.read_parquet('data/US/test_cash_flow.pq')
income_statement = pd.read_parquet('data/US/test_income_statement.pq')
GICS = pd.read_parquet('data/US/test_GICS.pq')
P = pd.read_parquet('data/US/test_adv_table.pq')
P.index.names = ['Date']
R = market_data['close'].unstack().reindex_like(P).pct_change()
market_data.dropna().sample(5)
fundamental_features = pd.read_parquet('data/US/fundamental_features.pq')
earnings = pd.read_parquet('data/US/test_earnings.pq')

C:\Users\marti\AppData\Local\Temp\ipykernel_112892\737443192.py:8: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  R = market_data['close'].unstack().reindex_like(P).pct_change()


In [ ]:
cash_flow_cols = cash_flow.columns.tolist()
# cash_flow_cols = [x.lower() for x in cash_flow_cols]
income_statement_cols  =income_statement.columns.tolist()

# income_statement_cols = [x.lower() for x in income_statement_cols]
balance_sheet_cols = balance_sheet.columns.tolist()
# balance_sheet_cols = [x.lower() for x in balance_sheet_cols]
def find_col(string_to_look:str,cash_flow_cols= cash_flow_cols,balance_sheet_cols = balance_sheet_cols,
             income_statement_cols= income_statement_cols):
    
    cash_flow_match = [ x for x in cash_flow_cols if string_to_look in x.lower()]
    if len(cash_flow_match) >0 :
        print('cash flow')
        print(cash_flow_match)
        
    income_statement_match = [ x for x in income_statement_cols if string_to_look in x.lower()]
    if len(income_statement_match) >0 :
        print('income_stateement')
        print(income_statement_match)
        
    balance_sheet_match = [ x for x in balance_sheet_cols if string_to_look in x.lower()]
    if len(balance_sheet_match) >0 :
        print('balance_sheet')
        print(balance_sheet_match)
    
find_col('exp')
    

In [4]:
fcf = cash_flow['freeCashFlow'].unstack().reindex_like(P).ffill().astype(float).fillna(0)
retained_earnings = balance_sheet['retainedEarnings'].unstack().reindex_like(P).ffill().astype(float).fillna(0)
working_capital = balance_sheet['netWorkingCapital'].unstack().reindex_like(P).ffill().astype(float).fillna(0)
total_assets = balance_sheet['totalAssets'].unstack().reindex_like(P).ffill().astype(float).fillna(0)
shares = balance_sheet['commonStockSharesOutstanding'].unstack().reindex_like(P).ffill().astype(float).fillna(0)
close = market_data['close'].unstack().reindex_like(P).ffill().astype(float).fillna(0)
total_liab = balance_sheet['totalLiab'].unstack().reindex_like(P).ffill().astype(float).fillna(0)
sales = income_statement['operatingIncome'].unstack().reindex_like(P).ffill().astype(float).fillna(0)
net_income = cash_flow['netIncome'].unstack().reindex_like(P).ffill().astype(float).fillna(0)
operating_cash_flow = cash_flow['totalCashFromOperatingActivities'].unstack().reindex_like(P).ffill().astype(float).fillna(0)

In [9]:
earnings_report = pd.read_parquet('data/US/test_earnings.pq')
earnings_report['reportDate'] = pd.to_datetime(earnings_report['reportDate'])
earnings_report['earnDate'] = earnings_report['reportDate']
earnings_report = earnings_report.set_index('reportDate',append=True)
earnings_report = earnings_report.droplevel(0).swaplevel().sort_index(level=0)
earnings_report.index.names = ['Date','Ticker']
earnings_report= earnings_report.drop_duplicates(keep='first')
earnings_report = earnings_report[~earnings_report.index.duplicated(keep=	'first')]

In [10]:
market_features = pd.read_parquet('data/US/market_features.pq')

In [11]:
earnings = earnings_report['epsDifference'].unstack().reindex_like(P)[P]

BM  = ((total_assets - total_liab)/ (shares * close)).clip(0,10)
BM = BM[P]
BM = BM.rank(ascending=True,pct=True).clip(0,1)

ACC =  (net_income - fcf) / total_assets 
ACC = ACC[P].rank(ascending=True,pct=True).clip(0,1)


EAR = earnings_report['epsActual'].unstack().reindex_like(P).ffill().astype(float)
EAR = (EAR * shares)/ total_assets
EAR = EAR[P].rank(ascending=True,pct=True).clip(0,1)

In [12]:
returns = market_features['daily_returns'].unstack().reindex_like(P).clip(-0.1,0.1)

In [13]:

def get_data_ticker(ticker,earnings_data=earnings,
                    book_market = BM,accruals = ACC,
                    earnings_income = EAR,returns = returns):
        # try :
            
            earning_dates = earnings_data[ticker].dropna().index.tolist()
            results_ticker = pd.DataFrame(index = earning_dates,columns = ['BHSAR','EAR','SUE','BM','ACC'])
            for i,date in enumerate(earning_dates[:-1]):
                    next_date = earning_dates[i+1]
                # try :
                    stop_date = next_date
                    earnings_ticker = earnings_data[ticker].dropna().loc[:date].tail(14)
                    # print(earnings_ticker)
                    sue_q = (earnings_ticker.iloc[-1] / earnings_ticker.std()) / 10
                    bm_q = book_market[ticker].loc[:date].iloc[-1]
                    acc_q = accruals[ticker].loc[:date].iloc[-1]
                    bhsar_q = returns[ticker].loc[date:stop_date].clip(-0.1,0.1).fillna(0).cumsum().iloc[-1]
                    ear_q = earnings_income[ticker].loc[:date].iloc[-1]
                    results_ticker.loc[date] = [bhsar_q,ear_q,sue_q,bm_q,acc_q]
                # except :
                    # pass
        # except :
            # pass
            return results_ticker
        

In [14]:
data = []
tickers_passed = []
for ticker in tqdm(P.columns.tolist()):
    try :
        data_ticker = get_data_ticker(ticker)
        data.append(data_ticker)
        tickers_passed.append(ticker)
    except :
        print(ticker, 'failed')
        pass

  0%|          | 4/879 [00:01<05:29,  2.66it/s]C:\Users\marti\AppData\Local\Temp\ipykernel_112892\3475342872.py:14: RuntimeWarning: invalid value encountered in scalar divide
  sue_q = (earnings_ticker.iloc[-1] / earnings_ticker.std()) / 10
  1%|          | 7/879 [00:02<05:30,  2.63it/s]C:\Users\marti\AppData\Local\Temp\ipykernel_112892\3475342872.py:14: RuntimeWarning: invalid value encountered in scalar divide
  sue_q = (earnings_ticker.iloc[-1] / earnings_ticker.std()) / 10
  1%|▏         | 12/879 [00:03<03:22,  4.29it/s]C:\Users\marti\AppData\Local\Temp\ipykernel_112892\3475342872.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  sue_q = (earnings_ticker.iloc[-1] / earnings_ticker.std()) / 10
  1%|▏         | 13/879 [00:04<04:03,  3.55it/s]C:\Users\marti\AppData\Local\Temp\ipykernel_112892\3475342872.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  sue_q = (earnings_ticker.iloc[-1] / earnings_ticker.std()) / 10
  2%|▏         | 16/879 [00:04<0

In [15]:
res = pd.concat(data,keys = tickers_passed)
res = res.swaplevel().sort_index(level=0)
res.to_parquet('data/US/data_strat_earnings.pq')

C:\Users\marti\AppData\Local\Temp\ipykernel_112892\4198139110.py:1: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  res = pd.concat(data,keys = tickers_passed)


In [16]:
import statsmodels.api as sm 
res = res.replace({np.inf:np.nan}).dropna()
all_years = res.resample('Y',level=0).mean().index
all_years.freq = None
all_years = all_years.tolist()

In [25]:
def get_signal(X_train,Y_train,X_test,Y_test):
    X_train = sm.add_constant(X_train)
    X_test = sm.add_constant(X_test)
    for col in (X_train.columns.tolist()):
        X_train[col] = X_train[col].unstack().ffill().rank(axis=1,pct=True,ascending=True).clip(0.001,0.999).apply(norm.ppf).stack().loc[X_train.index]
        X_test[col] = X_test[col].unstack().ffill().rank(axis=1,pct=True,ascending=True).clip(0.001,0.999).apply(norm.ppf).stack().loc[X_test.index]

    for col in (Y_train.columns.tolist()):
        Y_train[col] = Y_train[col].unstack().ffill().rank(axis=1,pct=True,ascending=True).clip(0.001,0.999).apply(norm.ppf).stack().loc[Y_train.index]
        Y_test[col] = Y_test[col].unstack().ffill().rank(axis=1,pct=True,ascending=True).clip(0.001,0.999).apply(norm.ppf).stack().loc[Y_test.index]


    # Fit the initial regression model
    model = sm.OLS(Y_train, X_train)
    results_ols = model.fit()
    results_ols.summary()
    pred = pd.DataFrame(results_ols.predict(X_test),columns = ['pred'])
    return pred

all_signal = []
for start_test in tqdm(all_years[4:]):
    start_train = start_test - pd.DateOffset(years =1)
    data_train = res.loc[start_train:start_test]
    data_test = res.loc[start_test:]
    X_train = sm.add_constant(data_train.drop('BHSAR',axis=1))
    X_test = sm.add_constant(data_test.drop('BHSAR',axis=1))
    Y_train = data_train[['BHSAR']]
    Y_test =data_test[['BHSAR']]
    pred = get_signal(X_train,Y_train,X_test,Y_test)
    all_signal.append(pred)
        

  0%|          | 0/20 [00:00<?, ?it/s]C:\Users\marti\AppData\Local\Temp\ipykernel_112892\170116002.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train[col] = Y_train[col].unstack().ffill().rank(axis=1,pct=True,ascending=True).clip(0.001,0.999).apply(norm.ppf).stack().loc[Y_train.index]
C:\Users\marti\AppData\Local\Temp\ipykernel_112892\170116002.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test[col] = Y_test[col].unstack().ffill().rank(axis=1,pct=True,ascending=True).clip(0.001,0.999).appl

AttributeError: 'Series' object has no attribute 'stack'

In [26]:
signal = pd.concat(all_signal,axis=0)[['pred']]
signal = signal[~signal.index.duplicated(keep='last')]
signal = signal['pred'].unstack().reindex_like(P).ffill()[P]

signal = signal[P].groupby(GICS['gicgrp'],axis=1).apply(center)
signal = signal[P].div(signal[P].abs().sum(1),0)

PNL = (signal[P].shift()*R.sub(R.mean(1),0)).dropna(axis=0,how='all').sum(1).dropna()
sr = sharpe(PNL)
turnover = signal.fillna(0).diff().abs().sum(1).mean()
PNL.cumsum().plot()
pd.Series({'Sharpe':sr,'Daily turnover (%)':turnover *100,'biais (bp)':PNL.mean()*1e4}).round(1)

In [53]:
%load_ext autoreload 
%autoreload 
from utils.func import *
test = aggregate_dividend_data(P.columns.tolist()[:10])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 10/10 [00:25<00:00,  2.60s/it]


In [52]:
test.sort_index(level=0).xs('AAPL',level=1).loc['2000':]

,declarationDate,recordDate,paymentDate,period,value,unadjustedValue,currency
Date,,,,,,,
2012-08-09,2012-07-24,2012-08-13,2012-08-16,Quarterly,0.0946,2.6488,USD
2012-11-07,2012-10-25,2012-11-12,2012-11-15,Quarterly,0.0946,2.6488,USD
2013-02-07,2013-01-23,2013-02-11,2013-02-14,Quarterly,0.0946,2.6488,USD
2013-05-09,2013-04-23,2013-05-13,2013-05-16,Quarterly,0.1089,3.0492,USD
2013-08-08,2013-07-23,2013-08-12,2013-08-15,Quarterly,0.1089,3.0492,USD
2013-11-06,2013-10-28,2013-11-11,2013-11-14,Quarterly,0.1089,3.0492,USD
2014-02-06,2014-01-27,2014-02-10,2014-02-13,Quarterly,0.1089,3.0492,USD
2014-05-08,2014-04-23,2014-05-12,2014-05-15,Quarterly,0.1175,3.2900,USD
2014-08-07,2014-07-22,2014-08-11,2014-08-14,Quarterly,0.1175,0.4700,USD


In [44]:
test.xs('AAPL',level=1).dropna()

,declarationDate,recordDate,paymentDate,period,value,unadjustedValue,currency
Date,,,,,,,
2012-08-09,2012-07-24,2012-08-13,2012-08-16,Quarterly,0.0946,2.6488,USD
2012-11-07,2012-10-25,2012-11-12,2012-11-15,Quarterly,0.0946,2.6488,USD
2013-02-07,2013-01-23,2013-02-11,2013-02-14,Quarterly,0.0946,2.6488,USD
2013-05-09,2013-04-23,2013-05-13,2013-05-16,Quarterly,0.1089,3.0492,USD
2013-08-08,2013-07-23,2013-08-12,2013-08-15,Quarterly,0.1089,3.0492,USD
2013-11-06,2013-10-28,2013-11-11,2013-11-14,Quarterly,0.1089,3.0492,USD
2014-02-06,2014-01-27,2014-02-10,2014-02-13,Quarterly,0.1089,3.0492,USD
2014-05-08,2014-04-23,2014-05-12,2014-05-15,Quarterly,0.1175,3.2900,USD
2014-08-07,2014-07-22,2014-08-11,2014-08-14,Quarterly,0.1175,0.4700,USD


In [ ]:
test